### Project name: TimeSpent
### Author: MCKitchen

### Goal: Keeping track of the time that I've spent in a given country


In [2]:
# Importing a number of modules:
import pandas as pd
import numpy as np
import os
import folium
import datetime
# Loading in the data where the number of days is based on memory and recent trips:
country_count = pd.read_csv('DataJuly2019.csv', delimiter = ';', header = 0 )

# How many days abroad:
daysabroad = country_count.Days[country_count.COUNTRY !='Netherlands'].sum()

# How many days in the netherlands 
today = datetime.date.today()
dob = datetime.date(1985, 4, 14)
diff = today - dob
# Update the field 
country_count.Days.loc[(country_count.COUNTRY == 'Netherlands')] = (diff.days - daysabroad)

# Given the fact that I've lived in the Netherlands and India for quite sometime, all other 
# countries literally pale in comparison. So to have some nice color grading I'm going to take the 
# log10 of the days that I spend in a givne country:
country_count_log10 = country_count
country_count_log10['Days'] = np.where(country_count_log10['Days'] > 0, np.log10(country_count_log10['Days']), float('nan'))

# Finally I need the polygoons of all the different countries. At first I used the following file "countries.geojson"
# from https://github.com/datasets/geo-countries. The problem that I had with this file was that the main land of france
# was combined with the same polygoon as French Guinnea. While legally perhaps true I find it a bit weird that I would
# use it to plot the countries that I visited I would also plot a country in South America when I visited France.
# Therefore I had to manually tweak the geojson file in http://geojson.io/ where I basically redrew france and French
# Guinnea separately: 
country_geo = os.path.join('Country_map.geojson')

# Now that we have all the required data lets start with a background worldmap which is part of the Folium
# module:
world_map = folium.Map(location=[10, 0], zoom_start=1.6, tiles= 'cartodbpositron')

# Lets now add an extra layer where the fill color is based on the log10(days spent in that country)
folium.Choropleth(
    geo_data = country_geo,
    name ='world_map',
    data = country_count_log10,
    columns = ['COUNTRY', 'Days'],
    key_on = 'feature.properties.ADMIN',
    fill_color = 'GnBu',
    nan_fill_color = 'white',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'log10(Number of days spent) | White is not visited yet',
    text_color = 'white'
).add_to(world_map)

# Add a missing value color code 
# Lets plot it:
world_map.save('TravelMapMCKitchen.html')

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log10
